In [1]:
import os # Configure which GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import random
import torch

from utils.NN_utils import BeamPredictionModel, BlockPredictionModel, BestGainPredictionModel
from utils.NN_utils import BeamPredictionLSTMModel, BlockPredictionLSTMModel, BestGainPredictionLSTMModel
from utils.NN_utils import preprocess_data
from utils.options import args_parser
from utils.mox_utils import setup_seed, get_save_dirs, np2torch, save_NN_results
from utils.data_utils import get_prepared_dataset, prepare_dataset
from utils.beam_utils import generate_dft_codebook, beamIdPair_to_beamPairId, beamPairId_to_beamIdPair



In [2]:
setup_seed(20)
freq = 28e9
DS_start, DS_end = 800, 950
preprocess_mode = 2
pos_in_data = preprocess_mode==2
look_ahead_len = 10
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 4
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = np2torch(data_np[:,0,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = 4
num_beampair = M_t * M_r

beam_pred_model = BeamPredictionModel(feature_input_dim, num_bs, num_beampair).to(device)
block_pred_model = BlockPredictionModel(feature_input_dim, num_bs).to(device)
gain_pred_model = BestGainPredictionModel(feature_input_dim, num_bs).to(device)

beam_pred_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/beampred_lstm_valAcc93.01%_2025-04-29_20:55:34.pth'))
block_pred_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/blockpred_lstm_valAcc98.87%_2025-04-26_13:08:55.pth'))
gain_pred_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/gainpred_lstm_valMae2.06dB_2025-04-29_20:18:49.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_model.eval()
block_pred_model.eval()
gain_pred_model.eval()
with torch.no_grad():
    beam_pred = beam_pred_model(data).argmax(dim=-1)
    block_pred = block_pred_model(data).argmax(dim=-1)
    gain_pred = gain_pred_model.pred(data)


Using device:  cuda:4


RuntimeError: Error(s) in loading state_dict for BeamPredictionModel:
	Missing key(s) in state_dict: "shared_layers.0.fc1.weight", "shared_layers.0.bn1.weight", "shared_layers.0.bn1.bias", "shared_layers.0.bn1.running_mean", "shared_layers.0.bn1.running_var", "shared_layers.0.fc2.weight", "shared_layers.0.bn2.weight", "shared_layers.0.bn2.bias", "shared_layers.0.bn2.running_mean", "shared_layers.0.bn2.running_var", "shared_layers.0.downsample.0.weight", "shared_layers.0.downsample.1.weight", "shared_layers.0.downsample.1.bias", "shared_layers.0.downsample.1.running_mean", "shared_layers.0.downsample.1.running_var", "shared_layers.2.fc1.weight", "shared_layers.2.bn1.weight", "shared_layers.2.bn1.bias", "shared_layers.2.bn1.running_mean", "shared_layers.2.bn1.running_var", "shared_layers.2.fc2.weight", "shared_layers.2.bn2.weight", "shared_layers.2.bn2.bias", "shared_layers.2.bn2.running_mean", "shared_layers.2.bn2.running_var", "shared_layers.2.downsample.0.weight", "shared_layers.2.dow

In [5]:
beam_pred_model

BeamPredictionModel(
  (shared_layers): Sequential(
    (0): VectorResBlock(
      (fc1): Linear(in_features=258, out_features=128, bias=False)
      (bn1): BatchNorm1d(258, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=128, out_features=128, bias=False)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Linear(in_features=258, out_features=128, bias=False)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ReLU()
    (2): VectorResBlock(
      (fc1): Linear(in_features=128, out_features=64, bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=64, out_features=64, bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [ ]:
(block_pred == block_labels).to(torch.float).mean()

In [ ]:
np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()

In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

# LSTM

In [2]:
setup_seed(20)
freq = 28e9
DS_start, DS_end = 800, 950
preprocess_mode = 2
pos_in_data = preprocess_mode==2
look_ahead_len = 10
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 3
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = np2torch(data_np[:,:-1,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = 4
num_beampair = M_t * M_r

beam_pred_lstm_model = BeamPredictionLSTMModel(feature_input_dim, num_bs, num_beampair).to(device)
block_pred_lstm_model = BlockPredictionLSTMModel(feature_input_dim, num_bs).to(device)
gain_pred_lstm_model = BestGainPredictionLSTMModel(feature_input_dim, num_bs).to(device)

beam_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/beampred_lstm_valAcc93.01%_2025-04-29_20:55:34.pth'))
block_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/blockpred_lstm_valAcc98.87%_2025-04-26_13:08:55.pth'))
gain_pred_lstm_model.load_state_dict(torch.load('NN_result/200_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead10/models/gainpred_lstm_valMae2.06dB_2025-04-29_20:18:49.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_lstm_model.eval()
block_pred_lstm_model.eval()
gain_pred_lstm_model.eval()

Using device:  cuda:3


BestGainPredictionLSTMModel(
  (lstm_layers): LSTM(258, 128, batch_first=True)
  (shared_layers): Sequential()
  (output_heads): ModuleList(
    (0-3): 4 x Sequential(
      (0): VectorResBlock(
        (fc1): Linear(in_features=128, out_features=128, bias=False)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fc2): Linear(in_features=128, out_features=128, bias=False)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ReLU()
      (2): VectorResBlock(
        (fc1): Linear(in_features=128, out_features=128, bias=False)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fc2): Linear(in_features=128, out_features=128, bias=False)
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (3): ReL

In [3]:
for input_len in range(1,look_ahead_len+1):
    input_data = data[:10000,-input_len:,:]
    with torch.no_grad():
        beam_pred = beam_pred_lstm_model(input_data).argmax(dim=-1)
        block_pred = block_pred_lstm_model(input_data).argmax(dim=-1)
        gain_pred = gain_pred_lstm_model.pred(input_data)

    acc_block_pred = (block_pred == block_labels).to(torch.float).mean()
    acc_beam_pred = (beam_pred == best_beam_pair_index_torch).to(torch.float).mean()
    mae_gain_pred = np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()
    print('Input length: ', input_len)
    print(f'--Block prediction accuracy: {acc_block_pred.item():.4f}', )
    print(f'--Beam prediction accuracy: {acc_beam_pred.item():.4f}', )
    print(f'--Mean absolute error of gain prediction: {mae_gain_pred.item():.4f}', )

RuntimeError: The size of tensor a (10000) must match the size of tensor b (146373) at non-singleton dimension 0

In [ ]:
for input_len in range(1,look_ahead_len+1):
    input_data = data[:,-input_len:,:]
    with torch.no_grad():
        beam_pred = beam_pred_lstm_model(input_data).argmax(dim=-1)
        block_pred = block_pred_lstm_model(input_data).argmax(dim=-1)
        gain_pred = gain_pred_lstm_model.pred(input_data)

    acc_block_pred = (block_pred == block_labels).to(torch.float).mean()
    acc_beam_pred = (beam_pred == best_beam_pair_index_torch).to(torch.float).mean()
    mae_gain_pred = np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()
    print('Input length: ', input_len)
    print(f'--Block prediction accuracy: {acc_block_pred.item():.4f}', )
    print(f'--Beam prediction accuracy: {acc_beam_pred.item():.4f}', )
    print(f'--Mean absolute error of gain prediction: {mae_gain_pred.item():.4f}', )

Input length:  1
--Block prediction accuracy: 0.9791
--Beam prediction accuracy: 0.8902
--Mean absolute error of gain prediction: 2.6942
Input length:  2
--Block prediction accuracy: 0.9814
--Beam prediction accuracy: 0.8959
--Mean absolute error of gain prediction: 2.4839
Input length:  3
--Block prediction accuracy: 0.9818
--Beam prediction accuracy: 0.8965
--Mean absolute error of gain prediction: 2.4575


In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

(tensor(-66.5101, device='cuda:4'),
 tensor(-187.5919, device='cuda:4'),
 -64.07656,
 -180.0)

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

2.4202566

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

2.4469101

In [ ]:
(gain_pred.detach().cpu().numpy()-g_opt).reshape(-1)

array([ 0.90330505,  0.11447144, -0.89450836, ...,  2.6871262 ,
       -0.29328918,  1.3418427 ], dtype=float32)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist((gain_pred.detach().cpu().numpy()-g_opt).reshape(-1), bins=100)
plt.show()

NameError: name 'gain_pred' is not defined

In [ ]:
block_pred.shape, gain_pred.shape

(torch.Size([147903, 4]), torch.Size([147903, 4]))

In [14]:
block_pred_lstm_model(data).shape, gain_pred_lstm_model.pred(data).shape

(torch.Size([147903, 4, 2]), torch.Size([147903, 4]))

In [15]:
output_block_pred_lstm_model = block_pred_lstm_model(data)
output_gain_pred_lstm_model = gain_pred_lstm_model(data)

In [16]:
output_block_pred_lstm_model.shape, output_block_pred_lstm_model

(torch.Size([147903, 4, 2]),
 tensor([[[  6.6171,  -4.2506],
          [ -5.9450,   6.9206],
          [  7.7003,  -8.2714],
          [  5.5485,  -7.2104]],
 
         [[ 12.5596, -11.4817],
          [ 12.8745, -10.7068],
          [  6.1245,  -6.6015],
          [  1.3027,  -1.2121]],
 
         [[ 15.5200, -13.9335],
          [ 10.5807,  -7.5470],
          [  2.1299,  -2.1151],
          [-10.5589,  10.4850]],
 
         ...,
 
         [[ -4.5845,   4.5399],
          [  7.5996,  -6.2444],
          [  7.9857,  -8.7114],
          [  5.6381,  -6.6196]],
 
         [[ -5.2027,   5.1565],
          [  3.5624,  -3.5991],
          [ 10.5413, -11.7617],
          [  7.2885,  -8.7141]],
 
         [[ -6.5844,   6.5532],
          [  2.5709,  -2.5035],
          [ 11.3501, -12.4060],
          [  7.5030,  -9.0474]]], device='cuda:4', grad_fn=<StackBackward0>))

In [17]:
output_gain_pred_lstm_model.shape, output_gain_pred_lstm_model

(torch.Size([147903, 4]),
 tensor([[ 2.4703, -1.9943,  3.1746,  2.6520],
         [ 3.0827,  2.6464,  2.6034,  0.3447],
         [ 2.8247,  2.3920,  0.5945, -2.0076],
         ...,
         [-1.5296,  2.0523,  2.5126,  3.2083],
         [-1.9940,  1.7486,  2.3294,  3.1009],
         [-2.0009,  1.2814,  2.3381,  3.0026]], device='cuda:4',
        grad_fn=<CatBackward0>))

In [18]:
torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1).shape, torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1)

(torch.Size([147903, 4]),
 tensor([[0, 1, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 1],
         ...,
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0]], device='cuda:4'))

In [19]:
error_loc = ~(block_labels == torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1))

In [14]:
import numpy as np
1.8e6*np.log2(1+1.24e-12 * 1e3/(10**(-174/10) * 1.8e6))

13397911.096200531